# Predictive Coding
stough 202-

In this notebook we're going to look at a way to account for very simple spatial redundancy, leading to much better compressibility. In short, a prediction of a pixel's value as its left neighbor is usually pretty effective, leading to a more compressible signal than the original without any loss of information. Read on for more.

**Huffman**:
In our discussion of [entropy](./entropy_intro.ipynb) we noted the different kinds of redundancy that we might leverage or account for in order to compress an image. We accounted for **coding redundancy** by applying Huffman variable length encoding. Huffman leverages differences in the relative probability of certain pixel values over others (low entropy) to define an encoding scheme that minimizes the average number of bits needed to represent each pixel value. Huffman's efficiency is inversely proportional to the entropy implied by the histogram of the image:
- If the histogram tends toward uniform, entropy is high and Huffman coding will accomplish little.
- If the histogram is highly non-uniform, with for example a few large spikes, then entropy is low and Huffman coding will work well.

**Predictive coding** computes some derivation of the image: the value at every pixel is its difference with respect to the pixel to the left, with the first column not changing. We'll see that while this is a completely reversible function, such a predictive coded version of an image can be much more compressible that the original. 

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For spatial filtering/operations
from scipy.ndimage import (correlate,
                           convolve)
from scipy.stats import entropy

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

In [ ]:
I = plt.imread('../dip_pics/skyandsea.jpg')
vis_hists(I)
print(arr_info(I))

In the above, we're viewing the image in its original, human-readable form. Let's compute the entropy of the image.

In [ ]:
freq, bb = np.histogram(I.ravel(), bins = np.arange(257))

In [ ]:
entropy(freq)

In [ ]:
J = I[...,0]
vis_hists(J)

In [ ]:
h = np.array([-1, 1], ndmin=2).astype('int16')

In [ ]:
h

In [ ]:
arr_info(h)

In [ ]:
Jf = correlate(J.astype('int16'), h, mode='constant', cval=0)
arr_info(Jf)

In [ ]:
vis_hists(Jf)

In [ ]:
Jf[:5, :10]

In [ ]:
J[:5,:10]

In [ ]:
Jr = np.cumsum(Jf, axis=1)
vis_hists(Jr)

In [ ]:
freq, bins = np.histogram(J.ravel(), bins=np.arange(257))

In [ ]:
entropy(freq, base=2)

In [ ]:
ff, bb = np.histogram(Jf.ravel(), bins = np.arange(-255,257))

In [ ]:
entropy(ff, base=2)